#**Project 1. Graph Convolutional Prediction of Protein Interactions in Yeast**

- Link prediction problem on unweighted and undirected networks

- Yeast PPI network -> prediction of new protein-pretein interactions

In [9]:
# connect to google drive
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

########################################################################################
# SET WORKING DIRECTORY TO PROJECT FOLDER BEFORE RUNNING!!
wd = gdrive_root + '/project5/'
########################################################################################

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [10]:
# import libraries
import time

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import scipy.sparse as sp
import networkx as nx

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

# set random seed
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

# **Hyperparameters + helper functions**

In [11]:

# set hyperparameters
hp = {}
hp['learning_rate'] = 0.01
hp['epochs'] = 20
hp['hidden1'] = 32
hp['hidden2'] = 16
hp['dropout'] = 0.1

print(hp)

{'learning_rate': 0.01, 'epochs': 20, 'hidden1': 32, 'hidden2': 16, 'dropout': 0.1}


In [0]:
# functions for data processing

# load adjacency matrix
def load_data():
  g = nx.read_edgelist(wd + 'yeast.edgelist')
  # g: networkx Graph object
  adj = nx.adjacency_matrix(g)
  # adj: SciPy sparse matrix
  return adj
  

# normalize adjacency matrix
def preprocess_graph(adj):
  # convert adj to COO format
  adj = sp.coo_matrix(adj)
  # add sparse identity matrix
  # for considering recurrent interaction during convolution
  adj_ = adj + sp.eye(adj.shape[0])

  # node degree vector for normalization
  rowsum = np.array(adj_.sum(1))
  degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())

  # sparse normalized adjacency matrix
  adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
  return adj_normalized


# convert sparse matrix to sparse torch tensor
def sparse_to_tensor(sparse_mx):
  # convert to COO format
  if not sp.isspmatrix_coo(sparse_mx):
    sparse_mx = sparse_mx.tocoo()
  
  # get arguments for tensor initialization
  values = sparse_mx.data
  indices = np.vstack((sparse_mx.row, sparse_mx.col))
  v = torch.FloatTensor(values)
  i = torch.LongTensor(indices)
  shape = sparse_mx.shape
  
  # make sparse tensor
  sparse_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))
  return sparse_tensor

In [0]:
# functions for dataset construction

# converting sparse matrix
def sparse_to_tuple(sparse_mx):
  if not sp.isspmatrix_coo(sparse_mx):
    sparse_mx = sparse_mx.tocoo()
  # coords: rows are coordinates
  coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
  values = sparse_mx.data
  shape = sparse_mx.shape
  return coords, values, shape


# build test/val set with 2% positive links
def mask_test_edges(adj):
  # remove diagonal elements
  adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
  adj.eliminate_zeros()

  # convert to upper triangular matrix
  adj_triu = sp.triu(adj)
 
  # get coordinates of unique, non-recurrent edges
  adj_tuple = sparse_to_tuple(adj_triu)
  edges = adj_tuple[0]
  
  # get coordinates of all nonzero elements from raw adjacency matrix
  edges_all = sparse_to_tuple(adj)[0]
  
  # test/val set size
  num_test = int(np.floor(edges.shape[0] / 50.))
  num_val = int(np.floor(edges.shape[0] / 50.))

  # random index selection
  all_edge_idx = np.linspace(0,edges.shape[0],edges.shape[0],
                             endpoint=False).astype(int)
  np.random.shuffle(all_edge_idx)
  val_edge_idx = all_edge_idx[:num_val]
  test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
  
  # edge selection
  # array of coordinates
  test_edges = edges[test_edge_idx]
  val_edges = edges[val_edge_idx]
  train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

  # auxiliary function
  # a: 1D array of two ints, b: 2D array of coordinates
  def ismember(a, b):
    rows_close = np.all((a - b[:, None]) == 0, axis=-1)
    return np.any(rows_close)

  # randomly selected unique negative edges for test set
  test_edges_false = []

  # generate until negative set size same as positive set
  while len(test_edges_false) < len(test_edges):
    # number of random set to generate
    n_rnd = len(test_edges) - len(test_edges_false)
    # randomly generate edges
    rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
    idxs_i = rnd[:n_rnd]                                        
    idxs_j = rnd[n_rnd:]
    # append to negative set if...
    for i in range(n_rnd):
      idx_i = idxs_i[i]
      idx_j = idxs_j[i]
      # not a recurrent edge
      if idx_i == idx_j:
        continue
      # not in raw adjacency matrix
      if ismember([idx_i, idx_j], edges_all):
        continue
      if test_edges_false:
        # not already in negative edge list
        if ismember([idx_j, idx_i], np.array(test_edges_false)):
          continue
        if ismember([idx_i, idx_j], np.array(test_edges_false)):
          continue
      # append to negative set
      test_edges_false.append([idx_i, idx_j])

  # randomly selected unique negative edges for val set
  val_edges_false = []
  # generate until negative set size same as positive set
  while len(val_edges_false) < len(val_edges):
    # number of random set to generate
    n_rnd = len(val_edges) - len(val_edges_false)
    # randomly generate edges
    rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
    idxs_i = rnd[:n_rnd]                                        
    idxs_j = rnd[n_rnd:]
    # append to negative set if...
    for i in range(n_rnd):
      idx_i = idxs_i[i]
      idx_j = idxs_j[i]
      # not a recurrent edge
      if idx_i == idx_j:
        continue
      # not in train edge list
      if ismember([idx_i, idx_j], train_edges):
        continue
      if ismember([idx_j, idx_i], train_edges):
        continue
      # not in val edge list
      if ismember([idx_i, idx_j], val_edges):
        continue
      if ismember([idx_j, idx_i], val_edges):
        continue
      if val_edges_false:
        # not already in val negative edge list
        if ismember([idx_j, idx_i], np.array(val_edges_false)):
          continue
        if ismember([idx_i, idx_j], np.array(val_edges_false)):
          continue
      # append to negative set
      val_edges_false.append([idx_i, idx_j])

  # re-build sparse adj matrix
  data = np.ones(train_edges.shape[0])
  adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
  adj_train = adj_train + adj_train.T

  return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# **Pytorch implementation of GCN modules**

In [0]:
# dropout for sparse input
def dropout_sparse(x, p, num_nonzero_elems):
  # current pytorch sparse tensors does not allow indexing
  coo = x.clone().detach()._indices().numpy()
  drop_num = int(num_nonzero_elems*p)
  drop_idx = np.random.choice(coo.shape[1], drop_num, replace=False)
  drop_coo = coo[:,drop_idx].T
  x = x.to_dense()
  x[drop_coo[:,0],drop_coo[:,1]] = 0
  indices = torch.nonzero(x).t()
  values = x[indices[0], indices[1]]
  o = torch.sparse.FloatTensor(indices, values, x.size())*(1./(1-p))
  return o

# graph convolution layer
# works both for sparse and dense input
class GraphConvolution(nn.Module):
  def __init__(self, input_dim, output_dim, adj, dropout=0, sparse_input=False):
    super().__init__()
    print(f'GraphConvolution object initialized with input dim = {input_dim}, output dim = {output_dim}')
    self.sparse_input = sparse_input
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.p = dropout
    self.adj = adj # normalized adjacency matrix (sparse tensor)
    
    self.w = nn.Parameter(torch.empty(output_dim, input_dim)) # feature weight matrix (dense tensor)
    nn.init.xavier_normal_(self.w)
    self.relu = nn.ReLU() # non-linearity
    self.dropout = nn.Dropout(p=self.p) # dropout

  def forward(self, x):
    # input, output tensor shapes: (num_features, num_nodes)
    num_nodes = x.shape[1]
    if x.shape[0] != self.input_dim:
      raise RuntimeError("input feature dimension not matched to input_dim argument")

    # forward
    if self.sparse_input:
      x = dropout_sparse(x, p=self.p, num_nonzero_elems=x._values().shape[0])
    else:
      x = self.dropout(x)
    x = torch.transpose(x, 0, 1) # (num_nodes, input_dim)
    x = torch.mm(x, torch.transpose(self.w, 0, 1)) # (num_nodes, output_dim)
    x = torch.mm(self.adj, x) # (num_nodes, output_dim)
    x = torch.transpose(x, 0, 1) # (output_dim, num_nodes)
    o = self.relu(x)
    return o
  
# decoder layer for link prediction
class InnerProductDecoder(nn.Module):
  def __init__(self, input_dim, dropout=0):
    super().__init__()
    print(f'InnerProductDecoder object initialized with input dim = {input_dim}')
    self.input_dim = input_dim
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, x):
    # input tensor shape: (num_features, num_nodes)
    x = self.dropout(x)
    xT = torch.transpose(x, 0, 1) # (num_nodes, input_dim)
    x = torch.mm(xT, x) # (num_nodes, num_nodes)
    o = torch.flatten(x)
    return o

In [0]:
class GCNModel(nn.Module):
  def __init__(self, input_dim, adj):
    super().__init__()
    print(f'GCNModel object initialized with input dim = {input_dim}')
    self.input_dim = input_dim
    self.hp = hp # hyperparameters
    self.adj = adj # normalized adjacency matrix

    self.gc1 = GraphConvolution(input_dim=self.input_dim,
                                output_dim=self.hp['hidden1'],
                                adj=self.adj,
                                dropout=self.hp['dropout'],
                                sparse_input=True)
    self.gc2 = GraphConvolution(input_dim=self.hp['hidden1'],
                                output_dim=self.hp['hidden2'],
                                adj=self.adj,
                                dropout=self.hp['dropout'],
                                sparse_input=False)
    self.dec = InnerProductDecoder(input_dim=self.hp['hidden2'],
                                   dropout=self.hp['dropout'])
    
  def forward(self, x):
    # input tensor shape: (input_dim, num_nodes)
    x = self.gc1(x)
    x = self.gc2(x)
    o = self.dec(x)
    # output tensor shape: (num_nodes*num_nodes, )
    return o

# **Main training routine**

In [0]:
# load adjacency matrix
adj = load_data() # raw adjacency matrix
num_nodes = adj.shape[0]
num_edges = np.sum(adj)

# train / val / test data
adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)

In [17]:
# input feature: none (node index one-hot)
features = sp.identity(num_nodes) # coords, values, shape
num_features = features.shape[1]

# normalized train adjacency matrix for graph convolution
adj_norm = preprocess_graph(adj_train)
adj_tensor = sparse_to_tensor(adj_norm)
adj_tensor.requires_grad = False

# initialize model
model = GCNModel(input_dim=num_features, adj=adj_tensor)

# set optimizer
optimizer = optim.Adam(model.parameters(), lr=hp['learning_rate'])

# define loss
def compute_loss(output, labels, num_nodes, num_edges):
  pos_weight = torch.ones([num_nodes*num_nodes])*float(num_nodes**2 - num_edges) / num_edges
  norm = num_nodes**2 / float((num_nodes**2 - num_edges) * 2)
  criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='mean')
  loss = norm*criterion(output.view((1, -1)),
                        labels.view((1, -1)))
  return loss

# evaluation metric
def get_roc_score(edges_pos, edges_neg, adj_orig, adj_label):
  # forward pass
  adj_rec = model(feat_tensor).view(adj_orig.shape).detach().numpy()

  def sigmoid(x):
    return 1 / (1 + np.exp(-x))

  # prediction of the model on given edge set
  preds_pos = []
  pos = []
  for e in edges_pos:
    preds_pos.append(sigmoid(adj_rec[e[0], e[1]]))
    pos.append(adj_orig[e[0], e[1]])
  
  preds_neg = []
  neg = []
  for e in edges_neg:
    preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
    neg.append(adj_orig[e[0], e[1]])

  preds_all = np.hstack([preds_pos, preds_neg])
  labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_pos))])
  roc_score = roc_auc_score(labels_all, preds_all)
  ap_score = average_precision_score(labels_all, preds_all)

  return roc_score, ap_score

GCNModel object initialized with input dim = 6526
GraphConvolution object initialized with input dim = 6526, output dim = 32
GraphConvolution object initialized with input dim = 32, output dim = 16
InnerProductDecoder object initialized with input dim = 16


In [18]:
# set input / target
feat_tensor = sparse_to_tensor(features)

# target label
adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = torch.flatten(sparse_to_tensor(adj_label).to_dense())
adj_label.requires_grad = False

# diagonal-removed adjacency matrix for ROC curve computation
adj_orig = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)

print('Preformance of initialized model')
roc_score, ap_score = get_roc_score(test_edges, test_edges_false, adj_orig, adj_label)
print('Initial ROC score: {:.5f}'.format(roc_score))
print('Initial AP score: {:.5f}'.format(ap_score))

for epoch in range(hp['epochs']):
  t = time.time()
  # set model to train mode
  model.train()
  # flush gradients
  optimizer.zero_grad()

  # forward pass
  output = model(feat_tensor)
  # compute loss
  loss = compute_loss(output=output, labels=adj_label, num_nodes=num_nodes, num_edges=num_edges)
  # backward pass
  loss.backward()
  print(f'\ncheck mean gradient: GC1 {np.mean(model.gc1.w.grad.clone().detach().numpy())}, GC2 {np.mean(model.gc2.w.grad.clone().detach().numpy())}')
  optimizer.step()
  
  # evaluate
  model.eval()
  roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false, adj_orig, adj_label)
  # log
  print("Epoch:", '%04d' % (epoch + 1),
        "train_loss=", "{:.5f}".format(loss),
        "val_roc=", "{:.5f}".format(roc_curr),
        "val_ap=", "{:.5f}".format(ap_curr),
        "time=", "{:.5f}".format(time.time() - t))

print('Optimization finished!')
roc_score, ap_score = get_roc_score(test_edges, test_edges_false, adj_orig, adj_label)
print('Test ROC score: {:.5f}'.format(roc_score))
print('Test AP score: {:.5f}'.format(ap_score))

Preformance of initialized model
Initial ROC score: 0.74474
Initial AP score: 0.65676

check mean gradient: GC1 -5.23995202783567e-09, GC2 -4.005696041531337e-08
Epoch: 0001 train_loss= 0.69499 val_roc= 0.87706 val_ap= 0.86806 time= 2.62946

check mean gradient: GC1 -2.9425973480101675e-07, GC2 -6.730005225108471e-06
Epoch: 0002 train_loss= 0.69475 val_roc= 0.87835 val_ap= 0.86890 time= 2.24086

check mean gradient: GC1 -7.717378025517974e-07, GC2 -3.844379534712061e-05
Epoch: 0003 train_loss= 0.69343 val_roc= 0.87828 val_ap= 0.86882 time= 2.24154

check mean gradient: GC1 -1.450709987693699e-06, GC2 -0.00011235527927055955
Epoch: 0004 train_loss= 0.69058 val_roc= 0.87817 val_ap= 0.86880 time= 2.28471

check mean gradient: GC1 -2.2823173821961973e-06, GC2 -0.000237693777307868
Epoch: 0005 train_loss= 0.68565 val_roc= 0.87816 val_ap= 0.86875 time= 2.23037

check mean gradient: GC1 -3.163746669088141e-06, GC2 -0.000403517740778625
Epoch: 0006 train_loss= 0.67897 val_roc= 0.87816 val_ap= 